In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import avg, lag

# Load Silver
df_silver = spark.read.format("delta").load("/delta/silver/machine_failure_clean")

# Define window per machine
w = Window.partitionBy("machine_id").orderBy("timestamp")

df_gold = df_silver \
    .withColumn("temp_roll_avg", avg("temperature").over(w.rowsBetween(-5, 0))) \
    .withColumn("temp_diff", col("temperature") - lag("temperature").over(w)) \
    .withColumn("pressure_roll_avg", avg("pressure").over(w.rowsBetween(-5, 0))) \
    .withColumn("vibration_roll_avg", avg("vibration").over(w.rowsBetween(-5, 0)))

# Save as Gold
df_gold.write.format("delta").mode("overwrite").save("/delta/gold/machine_features")
